# Team
Manan Oza

Sachin Venugopal

# Proximal Policy Optimization on CartPole-v0 and CartPole-v1

## 1.1 Motivation

#### The motivation of this project is to train a model to learn to play Cartpole-v0 using the Proximal Policy Optimization technique and to show the significant improvement in performance over 2 other commonly used algorithms. 

## 1.2 Proximal Policy Optimization

Policy Gradient methods have convergence problem which is addressed by the natural policy gradient. To improve training stability, we should avoid parameter updates that change the policy too much in one step. The Trust Region Policy Optimization and Proximal Policy Optimization are techniques that carry out this idea by enforcing a KL divergence constraint on the size of the policy update at each iteration. 

{Reference: https://spinningup.openai.com/en/latest/algorithms/ppo.html, https://medium.com/@jonathan_hui/rl-proximal-policy-optimization-ppo-explained-77f014ec3f12}

### Trust Region 

In the trust region method, we determine the maximum step size that we want to explore first. Then we locate the optimal point within the trust region and resume the search from there. In order not to make bad decisions, we can shrink the trust region if the policy is changing too much. Where TRPO tries to solve this problem with a complex second-order method, PPO is a family of first-order methods that use a few other tricks to keep new policies close to old. PPO methods are significantly simpler to implement, and empirically seem to perform at least as well as TRPO.



There are two primary variants of PPO:
   1. $\textbf{PPO-Penalty}$ approximately solves a KL-constrained update like TRPO, but penalizes the KL-divergence in the objective function instead of making it a hard constraint, and automatically adjusts the penalty coefficient over the course of training so that it’s scaled appropriately.
   2. $\textbf{PPO-Clip} $ doesn’t have a KL-divergence term in the objective and doesn’t have a constraint at all. Instead relies on specialized clipping in the objective function to remove incentives for the new policy to get far from the old policy.

PPO algorithm is as follows:

<img src="algo.png">
*Image taken from lecture slides.


### KL Divergence 

The Kullback-Leibler (KL) Divergence score quantifies how much one probability distribution differs from another probability distribution. In PPO, we limit how far we can change our policy in each iteration through the KL-divergence. 

> The KL Divergence between two distributions Q and P is :
    $ D_{KL} (P||Q) = E_{x}log\dfrac{P(x)}{Q(x)}$

We re-purpose it in PPO to measure the difference between any two policies. We don't want the new policy to be too different from the current one.


### Cartpole-v0
This environment consists of a pole attached to a cart which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright and the goal is to to prevent it from falling over( more than 15 degrees from vertical) while making sure that the cart does not move more than 2.4 units from the center. A reward of +1 is provided for every timestep the pole remains upright. The environment times out and terminates at the end of 200 timesteps.


    Actions: [0 1] where 0 => Move left 1 => Move right

    Observations: [0 1 2 3] where 0 => cart position, min = -2.4, max = -2.4 1 => Cart Velocity, min = -inf, max = inf 2 => Pole 

    Angle, min ~-41.8 degrees, max = ~41.8 degrees 3 => Pole Velocity At tip, min = -inf, max = inf

    Reward: +1 for every time step the pole is vertical, max = 200
    The stopping condition is a moving 100 episode reward average of 195
    
### Cartpole-v1
This environment consists of a pole attached to a cart which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright and the goal is to to prevent it from falling over( more than 15 degrees from vertical) while making sure that the cart does not move more than 2.4 units from the center. A reward of +1 is provided for every timestep the pole remains upright. The environment times out and terminates at the end of 500 timesteps.


    Actions: [0 1] where 0 => Move left 1 => Move right

    Observations: [0 1 2 3] where 0 => cart position, min = -2.4, max = -2.4 1 => Cart Velocity, min = -inf, max = inf 2 => Pole 

    Angle, min ~-41.8 degrees, max = ~41.8 degrees 3 => Pole Velocity At tip, min = -inf, max = inf

    Reward: +1 for every time step the pole is vertical, max = 500
    The stopping condition is a moving 100 episode reward average of 475

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense
from keras import backend as K
from keras.optimizers import Adam
import gym
import matplotlib.pyplot as plt

In [ ]:
### Here we define the PPO loss function that we will use while compliling the actor model.
### This loss function calculates the clipped advantage value and adds the clipped entropy.
### The entropy helps in regulating exploitation and exploration.
### The agents gets penalized for lesser exploration.

def ppo_loss_func(advantage, prev_pred, clip_loss = 0.2, clip_entropy = 5e-3):
    def loss(y_true, y_pred):
        ratio = K.sum(y_true * y_pred, axis = -1) / (K.sum(y_true * prev_pred, axis = -1) + 1e-7)
        probability = K.sum(y_true * y_pred, axis = -1)
        clipped_loss = -K.mean(K.minimum(ratio * advantage, K.clip(ratio, min_value = (1 - clip_loss), 
                    max_value = (1 + clip_loss)) * advantage) + clip_entropy * -(probability
                    * K.log(probability + 1e-7)))
        return clipped_loss
    return loss

In [ ]:
### Agent Class
class Agent():
    def __init__(self, env, action_space, state_space, learning_rate, buffer_len, gamma):
        self.env = env
        self.reward = []
        self.observation = self.env.reset()
        self.learning_rate = learning_rate ### learning rate
        self.buffer_len = buffer_len ### The length of buffer which stores previous results from which we 
        self.gamma = gamma  ### discount factor
        self.action_space = action_space
        self.state_space = state_space
        self.actor = self.build_actor_network()
        self.critic = self.build_critic_network()
    
    def build_actor_network(self): ### builds the actor network
        input_state = Input(shape=(self.state_space,))
        advantage = Input(shape=(1,))
        pred = Input(shape=(self.action_space,))
        
        layer_1 = Dense(128, activation='tanh')(input_state)
        layer_2 = Dense(128, activation='tanh')(layer_1)
        out = Dense(self.action_space, activation='softmax')(layer_2)
        
        actor_model = Model(inputs=[input_state, advantage, pred], outputs=[out])
        actor_model.compile(optimizer=Adam(lr=self.learning_rate), 
                      loss=[ppo_loss_func(advantage=advantage, prev_pred=pred)]) ### here we use the 
                                                                            ### loss function defined by us earlier
        
        actor_model.summary()
        return actor_model
    
    def build_critic_network(self): ### builds the critic network
        input_state = Input(shape=(self.state_space,))
        
        layer_1 = Dense(128, activation='tanh')(input_state)
        layer_2 = Dense(128, activation='tanh')(layer_1)
        out = Dense(1)(layer_2)
        
        critic_model = Model(inputs=[input_state], outputs=[out])
        critic_model.compile(optimizer=Adam(lr=self.learning_rate), loss='mse')
        
        critic_model.summary()
        return critic_model
    
    def discounted_reward(self):
        a = np.array(self.reward).sum()
        for t in range(len(self.reward) - 2, -1, -1):
            self.reward[t] += self.reward[t + 1] * self.gamma ### multiplies gamma^(T-t+1) with reward[t+1]
                                                              ### i.e. discounted reward
        return a
    
    def fetch_act(self):
        predicted_action = self.actor.predict([self.observation.reshape(1, 4), 
                                               np.zeros((1, 1)), np.zeros((1, 2))]) ### predicts which action 
                                                            ### needs to be taken based in the current observation
        action_taken = np.random.choice(2, p=np.nan_to_num(predicted_action[0])) ### the performed action
        action_mat = np.zeros(2) ### makes a matrix of two '0' values and assigns '1' to the 
                                ### corresponding action that has been performed
        action_mat[action_taken] = 1
        return action_taken, action_mat, predicted_action
    
    def get_batch(self): ### used to make batches
        batch = [[], [], [], []]

        per_episode_reward = []
        observation_list, action_list, pred_list = [], [], []
        while len(batch[0]) < self.buffer_len:
            action, action_matrix, predicted_action = self.fetch_act()
            observation, reward, done, info = self.env.step(action) ### Performs the said action
#             self.env.render()  ### Uncomment to render the environment while training
            self.reward.append(reward)

            observation_list.append(self.observation) ### list of all observations
            action_list.append(action_matrix) ### list of all actions performed
            pred_list.append(predicted_action) ### list of all predicted actions
            self.observation = observation
            
            ### This for loop creates batches of the observations, actions taken, 
            ### predicted actions and the corresponding rewards
            ### These batches are then used to train the actor and critic networks.
            if done == True:
                per_episode_reward.append(self.discounted_reward())
                for i in range(len(self.reward)):
                    obs_batch = observation_list[i]
                    action_batch = action_list[i]
                    pred_batch = pred_list[i]
                    reward_batch = self.reward[i]
                    batch[0].append(obs_batch)
                    batch[1].append(action_batch)
                    batch[2].append(pred_batch)
                    batch[3].append(reward_batch)
                observation_mat, action_mat, pred_mat = [], [], []
                self.observation = self.env.reset()
                self.reward = []
        
        obs_array = np.array(batch[0])
        action_array = np.array(batch[1])
        predict_array = np.array(batch[2])
        reward_array = np.reshape(np.array(batch[3]), (len(batch[3]), 1))
        
        predict_array = np.reshape(predict_array, (predict_array.shape[0], predict_array.shape[2]))
        return obs_array, action_array, predict_array, reward_array, per_episode_reward ### returns the batches 
                                                                                ### in the form of numpy arrays


    def train(self, max_episodes, epochs, batch_size): ### trainer function
        reward_list = []
        actor_history_list = []
        critic_history_list = []
        moving_avg = []
        
        for episode in range(max_episodes):
            obs, action, pred, reward, per_episode_reward = self.get_batch() ### generates a batch of observations,
                                                                    ### actions, predicted actions and rewards
            print("Episode = " + str(episode) + " || average reward = " + str(np.mean(per_episode_reward)))
            obs = obs[:self.buffer_len]
            pred = pred[:self.buffer_len]
            action = action[:self.buffer_len]
            reward = reward[:self.buffer_len]
            
            prev_pred = pred
            state_values = self.critic.predict(obs) ### gets output from critic (state values)
            advantage = reward - state_values ### calculates the advantage
            
            reward_list.append(np.mean(per_episode_reward))
            if episode <= 99:
                moving_avg.append(0)
            else:
                moving_avg.append(np.mean(reward_list))
            
            ### trains the actor network for epochs = 10
            self.actor_loss = self.actor.fit([obs, advantage, prev_pred], [action], 
                            batch_size = batch_size, shuffle=True, epochs=epochs, verbose = 0)
            
            ### trains the critic network for epochs = 10
            self.critic_loss = self.critic.fit([obs], [reward], 
                                batch_size=batch_size, shuffle=True, epochs=epochs, verbose = 0)
            
            actor_history_list = actor_history_list + self.actor_loss.history['loss']
            critic_history_list = critic_history_list + self.critic_loss.history['loss']
        
        ### plots rewards vs episodes
        plt.plot(reward_list)
        plt.plot(moving_avg)
        plt.title('Rewards')
        plt.ylabel('Reward per episode')
        plt.xlabel('Episode')
        plt.legend(['Per episode reward', '100 episode moving reward'], loc='upper left')
        plt.show()
        
        ### plots loss for actor network
        plt.plot(actor_history_list)
        plt.title('Actor loss')
        plt.ylabel('loss')
        plt.xlabel('Epoch')
        plt.legend(['Train'], loc='upper left')
        plt.show()
        
        ### plots loss for critic network
        plt.plot(critic_history_list)
        plt.title('Critic loss')
        plt.ylabel('loss')
        plt.xlabel('Epoch')
        plt.legend(['Train'], loc='upper left')
        plt.show()

In [ ]:
def main(env):
    env = gym.make(env)
    max_episodes = 1000
    epochs = 10 ### train the networks for 10 epochs at one time.
    gamma = 0.99
    buffer_len = 2048
    batch_size = 256
    action_space = env.action_space.n
    state_space = env.observation_space.shape[0]
    learning_rate = 1e-4
    agent = Agent(env, action_space, state_space, learning_rate, buffer_len, gamma)
    agent.train(max_episodes, epochs, batch_size)

In [ ]:
if __name__ == '__main__':
    env = "CartPole-v1"
    main(env)

# Execute this to test the trained models

In [ ]:
from keras.layers import Dense, Input
from keras.models import Model, load_model
from keras.optimizers import Adam
import numpy as np
import random
import argparse
import gym
import keras 
from gym import wrappers, logger
# Test the PPO trained model for CartPole-v0
env = gym.make('CartPole-v0') ### change to 'CartPole-v1' 
env.reset()
model = load_model("cartpole_v0_ppo") ### change file according to the environment
state = env.reset()
done = False
final_reward = 0
while not Done: # can be set to while True for an infnitely long episode
    env.render()
    state = np.expand_dims(np.array(state), axis=0)
    action = model.predict(state)
    next_state, reward, done, info = env.step(action=np.argmax(action))
    state = next_state
    final_reward += reward
env.close()
print(final_reward)

# RESULTS
## PPO Losses
#### CartPole-v0
<img src="ppocartpolev0losses.png">

#### CartPole-v1
<img src="ppocartpolev1losses.png">

# Comparing with DDQN and DQN
## CartPole-v0
### PPO
<img src="ppocartpolev0rewards.png">

### DDQN
<img src="ddqncartpole0.png">

### DQN
<img src="dqncartpolev0.png">

## CartPole-v1
### PPO
<img src="ppocartpolev1rewards.png">

### DDQN
<img src="ddqncartpolev1.png">

### DQN
<img src="dqncartpolev1.png">


Looking at the above graphs of rewards vs episodes we can easily conclude that PPO gives us better training results and also makes the training stable as compared to DDQN and DQN. The PPO agent learns faster than the other two algorithms and also consistently learns whereas the DDQN and DQN agents have a noisy learning curve and hence the plot of rewards are quite noisy. The PPO algorithm is therefore a much better learning algorithm that gives robust results.